##**Setup the Environment**

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from keras.models import Model

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np


##**Download Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/MyDrive/DS340/nutrition5k/'
!cp -r /content/drive/MyDrive/DS340/nutrition5k /content/

In [9]:

csv_path = '/content/drive/MyDrive/DS340/df_merged.csv'
df_data = pd.read_csv(csv_path)
df_data

,Filepath,sideAPath,sideBPath,sideCPath,sideDPath,calories,mass,fat,carb,protein
0,nutrition5k/imagery/overhead/dish_1561662054/r...,nutrition5k/imagery/sideA/dish_1561662054/firs...,nutrition5k/imagery/sideB/dish_1561662054/firs...,nutrition5k/imagery/sideC/dish_1561662054/firs...,nutrition5k/imagery/sideD/dish_1561662054/firs...,419.438782,292.0,23.838249,26.351543,25.910593
1,nutrition5k/imagery/overhead/dish_1562008979/r...,nutrition5k/imagery/sideA/dish_1562008979/firs...,nutrition5k/imagery/sideB/dish_1562008979/firs...,nutrition5k/imagery/sideC/dish_1562008979/firs...,nutrition5k/imagery/sideD/dish_1562008979/firs...,382.936646,290.0,22.224644,10.173570,35.345387
2,nutrition5k/imagery/overhead/dish_1560455030/r...,nutrition5k/imagery/sideA/dish_1560455030/firs...,nutrition5k/imagery/sideB/dish_1560455030/firs...,nutrition5k/imagery/sideC/dish_1560455030/firs...,nutrition5k/imagery/sideD/dish_1560455030/firs...,20.590000,103.0,0.148000,4.625000,0.956000
3,nutrition5k/imagery/overhead/dish_1558372433/r...,nutrition5k/imagery/sideA/dish_1558372433/firs...,nutrition5k/imagery/sideB/dish_1558372433/firs...,nutrition5k/imagery/sideC/dish_1558372433/firs...,nutrition5k/imagery/sideD/dish_1558372433/firs...,74.360001,143.0,0.286000,0.429000,20.020000
4,nutrition5k/imagery/overhead/dish_1565640549/r...,nutrition5k/imagery/sideA/dish_1565640549/firs...,nutrition5k/imagery/sideB/dish_1565640549/firs...,nutrition5k/imagery/sideC/dish_1565640549/firs...,nutrition5k/imagery/sideD/dish_1565640549/firs...,45.482903,139.0,1.568471,7.043886,2.641478
...,...,...,...,...,...,...,...,...,...,...
218,nutrition5k/imagery/overhead/dish_1563220987/r...,nutrition5k/imagery/sideA/dish_1563220987/firs...,nutrition5k/imagery/sideB/dish_1563220987/firs...,nutrition5k/imagery/sideC/dish_1563220987/firs...,nutrition5k/imagery/sideD/dish_1563220987/firs...,224.987503,110.0,12.479591,4.072388,22.457699
219,nutrition5k/imagery/overhead/dish_1563913709/r...,nutrition5k/imagery/sideA/dish_1563913709/firs...,nutrition5k/imagery/sideB/dish_1563913709/firs...,nutrition5k/imagery/sideC/dish_1563913709/firs...,nutrition5k/imagery/sideD/dish_1563913709/firs...,337.888184,263.0,13.284250,42.313908,13.815399
220,nutrition5k/imagery/overhead/dish_1563305883/r...,nutrition5k/imagery/sideA/dish_1563305883/firs...,nutrition5k/imagery/sideB/dish_1563305883/firs...,nutrition5k/imagery/sideC/dish_1563305883/firs...,nutrition5k/imagery/sideD/dish_1563305883/firs...,83.052505,77.0,7.017997,4.055908,2.147450
221,nutrition5k/imagery/overhead/dish_1560454488/r...,nutrition5k/imagery/sideA/dish_1560454488/firs...,nutrition5k/imagery/sideB/dish_1560454488/firs...,nutrition5k/imagery/sideC/dish_1560454488/firs...,nutrition5k/imagery/sideD/dish_1560454488/firs...,48.244003,42.0,4.500726,1.893843,0.635490


In [10]:
df_data['sideAPath'][0]

'nutrition5k/imagery/sideA/dish_1561662054/first_frame_camera_A.png'

##**Adding more data**

In [ ]:
!sudo apt install -y gcc g++ gfortran
!pip install --upgrade gsutil

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
gfortran is already the newest version (4:11.2.0-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.1/178.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━

In [ ]:
import pandas as pd

# get dataset
df = pd.read_csv('nutrition5k/metadata/dish_metadata_cafe1.csv', usecols=range(6),  nrows = 200)
df.columns = ['id', 'calories', 'mass', 'fat', 'carb', 'protein']

print(df)

                  id    calories   mass        fat       carb    protein
0    dish_1562688426  137.569992   88.0   8.256000   5.190000  10.297000
1    dish_1561662054  419.438782  292.0  23.838249  26.351543  25.910593
2    dish_1562008979  382.936646  290.0  22.224644  10.173570  35.345387
3    dish_1560455030   20.590000  103.0   0.148000   4.625000   0.956000
4    dish_1558372433   74.360001  143.0   0.286000   0.429000  20.020000
..               ...         ...    ...        ...        ...        ...
195  dish_1561665024  100.570984  121.0   6.984664   8.948565   2.141329
196  dish_1550710448   48.910000   67.0   3.216000   1.273000   5.360000
197  dish_1563813308  224.982895   76.0  16.446194   3.901661  15.332276
198  dish_1551235600   83.659996  178.0   0.178000  21.359999   1.602000
199  dish_1561062555   47.093922   32.0   3.147569   4.405365   1.027732

[200 rows x 6 columns]


In [ ]:
import pandas as pd

# get dataset
df = pd.read_csv('nutrition5k/metadata/dish_metadata_cafe1.csv', usecols=range(6), skiprows=range(0, 200), nrows = 200)
df.columns = ['id', 'calories', 'mass', 'fat', 'carb', 'protein']

print(df)

                  id    calories   mass        fat       carb     protein
0    dish_1558723414   51.988998   49.0   1.176000  10.780000    1.519000
1    dish_1562691674  376.976868  435.0  11.338250  42.573318   26.482393
2    dish_1566591080  909.629578  549.0  43.513371  19.378721  103.567314
3    dish_1550862648   16.119999   31.0   0.062000   4.340000    0.093000
4    dish_1564082462  905.354492  609.0  32.071827  40.532917  108.822510
..               ...         ...    ...        ...        ...         ...
195  dish_1558639335   60.320000  116.0   0.232000  16.240000    0.348000
196  dish_1560454488   48.244003   42.0   4.500726   1.893843    0.635490
197  dish_1559933793   56.676903  111.0   1.420199   9.842957    3.716384
198  dish_1563382953  249.919525  170.0  16.234716  12.476491   12.876415
199  dish_1550873264   97.349998   59.0   2.124000   0.000000   18.290001

[200 rows x 6 columns]


In [ ]:
import subprocess
import pandas as pd
import os

all_ids = df['id']
overhead_paths = []
sideA_paths = []
sideB_paths = []
sideC_paths = []
sideD_paths = []

def extract_first_frame(video_path, output_path):
    result = subprocess.run(
        ["ffmpeg", "-i", video_path, "-vf", "select=eq(n\,0)", "-vsync", "vfr", output_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    return result.returncode == 0

for i in all_ids:
    # Download overhead image
    file_path = f"gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/{i}/rgb.png"
    local_path = f"nutrition5k/imagery/overhead/{i}/"
    os.makedirs(local_path, exist_ok=True)

    result1 = subprocess.run(
        ["gsutil", "cp", file_path, local_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    if result1.returncode == 0:
        print(f"Succeed to download: {file_path}")
    else:
        print(f"Failed to download: {file_path}")
        continue

    # Download side video and extract first frame image
    def download_video_and_extract_first_frame(camera, local_video_path, local_image_path):
        video_path = f"gs://nutrition5k_dataset/nutrition5k_dataset/imagery/side_angles/{i}/{camera}.h264"
        os.makedirs(local_video_path, exist_ok=True)

        result = subprocess.run(
            ["gsutil", "cp", video_path, local_video_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        if result.returncode == 0:
            video_file = os.path.join(local_video_path, f"{camera}.h264")
            image_file = os.path.join(local_image_path, f"first_frame_{camera}.png")

            if extract_first_frame(video_file, image_file):
                #print(f"Succeed to extract first frame from {camera}: {video_path}")
                return image_file
            else:
                #print(f"Failed to extract first frame from {camera}: {video_path}")
                return None
        else:
            #print(f"Failed to download video from {video_path}")
            return None

    sideA_image_path = download_video_and_extract_first_frame('camera_A', f"nutrition5k/imagery/sideA/{i}", f"nutrition5k/imagery/sideA/{i}")
    if not sideA_image_path:
        continue
    sideB_image_path = download_video_and_extract_first_frame('camera_B', f"nutrition5k/imagery/sideB/{i}", f"nutrition5k/imagery/sideB/{i}")
    if not sideB_image_path:
        continue
    sideC_image_path = download_video_and_extract_first_frame('camera_C', f"nutrition5k/imagery/sideC/{i}", f"nutrition5k/imagery/sideC/{i}")
    if not sideC_image_path:
        continue
    sideD_image_path = download_video_and_extract_first_frame('camera_D', f"nutrition5k/imagery/sideD/{i}", f"nutrition5k/imagery/sideD/{i}")
    if not sideD_image_path:
        continue
    sideA_paths.append(sideA_image_path)
    sideB_paths.append(sideB_image_path)
    sideC_paths.append(sideC_image_path)
    sideD_paths.append(sideD_image_path)
    overhead_paths.append(os.path.join(local_path, 'rgb.png'))

new_df = pd.DataFrame({
    'Filepath': overhead_paths,
    'sideAPath': sideA_paths,
    'sideBPath': sideB_paths,
    'sideCPath': sideC_paths,
    'sideDPath': sideD_paths
})

new_df['id'] = new_df['Filepath'].apply(lambda x: x.split('/')[-2])

new_df = new_df.merge(df, on='id')
new_df = new_df[['Filepath', 'sideAPath', 'sideBPath', 'sideCPath', 'sideDPath', 'calories', 'mass', 'fat', 'carb', 'protein']]

Succeed to download: gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/dish_1558723414/rgb.png
Succeed to download: gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/dish_1562691674/rgb.png
Succeed to download: gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/dish_1566591080/rgb.png
Failed to download: gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/dish_1550862648/rgb.png
Succeed to download: gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/dish_1564082462/rgb.png
Succeed to download: gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/dish_1562602860/rgb.png
Failed to download: gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/dish_1568217372/rgb.png
Succeed to download: gs://nutrition5k_dataset/nutrition5k_dataset/imagery/realsense_overhead/dish_1561739954/rgb.png
Succeed to download: gs://nutrition5k_dataset/nutrition5k_dataset/

In [ ]:
new_df

,Filepath,sideAPath,sideBPath,sideCPath,sideDPath,calories,mass,fat,carb,protein
0,nutrition5k/imagery/overhead/dish_1562691674/r...,nutrition5k/imagery/sideA/dish_1562691674/firs...,nutrition5k/imagery/sideB/dish_1562691674/firs...,nutrition5k/imagery/sideC/dish_1562691674/firs...,nutrition5k/imagery/sideD/dish_1562691674/firs...,376.976868,435.0,11.338250,42.573318,26.482393
1,nutrition5k/imagery/overhead/dish_1566591080/r...,nutrition5k/imagery/sideA/dish_1566591080/firs...,nutrition5k/imagery/sideB/dish_1566591080/firs...,nutrition5k/imagery/sideC/dish_1566591080/firs...,nutrition5k/imagery/sideD/dish_1566591080/firs...,909.629578,549.0,43.513371,19.378721,103.567314
2,nutrition5k/imagery/overhead/dish_1564082462/r...,nutrition5k/imagery/sideA/dish_1564082462/firs...,nutrition5k/imagery/sideB/dish_1564082462/firs...,nutrition5k/imagery/sideC/dish_1564082462/firs...,nutrition5k/imagery/sideD/dish_1564082462/firs...,905.354492,609.0,32.071827,40.532917,108.822510
3,nutrition5k/imagery/overhead/dish_1562602860/r...,nutrition5k/imagery/sideA/dish_1562602860/firs...,nutrition5k/imagery/sideB/dish_1562602860/firs...,nutrition5k/imagery/sideC/dish_1562602860/firs...,nutrition5k/imagery/sideD/dish_1562602860/firs...,165.182281,258.0,5.622155,20.834251,10.206213
4,nutrition5k/imagery/overhead/dish_1561739954/r...,nutrition5k/imagery/sideA/dish_1561739954/firs...,nutrition5k/imagery/sideB/dish_1561739954/firs...,nutrition5k/imagery/sideC/dish_1561739954/firs...,nutrition5k/imagery/sideD/dish_1561739954/firs...,276.119995,225.0,16.059999,16.556000,15.390000
...,...,...,...,...,...,...,...,...,...,...
114,nutrition5k/imagery/overhead/dish_1563220987/r...,nutrition5k/imagery/sideA/dish_1563220987/firs...,nutrition5k/imagery/sideB/dish_1563220987/firs...,nutrition5k/imagery/sideC/dish_1563220987/firs...,nutrition5k/imagery/sideD/dish_1563220987/firs...,224.987503,110.0,12.479591,4.072388,22.457699
115,nutrition5k/imagery/overhead/dish_1563913709/r...,nutrition5k/imagery/sideA/dish_1563913709/firs...,nutrition5k/imagery/sideB/dish_1563913709/firs...,nutrition5k/imagery/sideC/dish_1563913709/firs...,nutrition5k/imagery/sideD/dish_1563913709/firs...,337.888184,263.0,13.284250,42.313908,13.815399
116,nutrition5k/imagery/overhead/dish_1563305883/r...,nutrition5k/imagery/sideA/dish_1563305883/firs...,nutrition5k/imagery/sideB/dish_1563305883/firs...,nutrition5k/imagery/sideC/dish_1563305883/firs...,nutrition5k/imagery/sideD/dish_1563305883/firs...,83.052505,77.0,7.017997,4.055908,2.147450
117,nutrition5k/imagery/overhead/dish_1560454488/r...,nutrition5k/imagery/sideA/dish_1560454488/firs...,nutrition5k/imagery/sideB/dish_1560454488/firs...,nutrition5k/imagery/sideC/dish_1560454488/firs...,nutrition5k/imagery/sideD/dish_1560454488/firs...,48.244003,42.0,4.500726,1.893843,0.635490


In [ ]:
df_merged = pd.concat([df_data, new_df], ignore_index=True)
df_merged

,Filepath,sideAPath,sideBPath,sideCPath,sideDPath,calories,mass,fat,carb,protein
0,nutrition5k/imagery/overhead/dish_1561662054/r...,nutrition5k/imagery/sideA/dish_1561662054/firs...,nutrition5k/imagery/sideB/dish_1561662054/firs...,nutrition5k/imagery/sideC/dish_1561662054/firs...,nutrition5k/imagery/sideD/dish_1561662054/firs...,419.438782,292.0,23.838249,26.351543,25.910593
1,nutrition5k/imagery/overhead/dish_1562008979/r...,nutrition5k/imagery/sideA/dish_1562008979/firs...,nutrition5k/imagery/sideB/dish_1562008979/firs...,nutrition5k/imagery/sideC/dish_1562008979/firs...,nutrition5k/imagery/sideD/dish_1562008979/firs...,382.936646,290.0,22.224644,10.173570,35.345387
2,nutrition5k/imagery/overhead/dish_1560455030/r...,nutrition5k/imagery/sideA/dish_1560455030/firs...,nutrition5k/imagery/sideB/dish_1560455030/firs...,nutrition5k/imagery/sideC/dish_1560455030/firs...,nutrition5k/imagery/sideD/dish_1560455030/firs...,20.590000,103.0,0.148000,4.625000,0.956000
3,nutrition5k/imagery/overhead/dish_1558372433/r...,nutrition5k/imagery/sideA/dish_1558372433/firs...,nutrition5k/imagery/sideB/dish_1558372433/firs...,nutrition5k/imagery/sideC/dish_1558372433/firs...,nutrition5k/imagery/sideD/dish_1558372433/firs...,74.360001,143.0,0.286000,0.429000,20.020000
4,nutrition5k/imagery/overhead/dish_1565640549/r...,nutrition5k/imagery/sideA/dish_1565640549/firs...,nutrition5k/imagery/sideB/dish_1565640549/firs...,nutrition5k/imagery/sideC/dish_1565640549/firs...,nutrition5k/imagery/sideD/dish_1565640549/firs...,45.482903,139.0,1.568471,7.043886,2.641478
...,...,...,...,...,...,...,...,...,...,...
218,nutrition5k/imagery/overhead/dish_1563220987/r...,nutrition5k/imagery/sideA/dish_1563220987/firs...,nutrition5k/imagery/sideB/dish_1563220987/firs...,nutrition5k/imagery/sideC/dish_1563220987/firs...,nutrition5k/imagery/sideD/dish_1563220987/firs...,224.987503,110.0,12.479591,4.072388,22.457699
219,nutrition5k/imagery/overhead/dish_1563913709/r...,nutrition5k/imagery/sideA/dish_1563913709/firs...,nutrition5k/imagery/sideB/dish_1563913709/firs...,nutrition5k/imagery/sideC/dish_1563913709/firs...,nutrition5k/imagery/sideD/dish_1563913709/firs...,337.888184,263.0,13.284250,42.313908,13.815399
220,nutrition5k/imagery/overhead/dish_1563305883/r...,nutrition5k/imagery/sideA/dish_1563305883/firs...,nutrition5k/imagery/sideB/dish_1563305883/firs...,nutrition5k/imagery/sideC/dish_1563305883/firs...,nutrition5k/imagery/sideD/dish_1563305883/firs...,83.052505,77.0,7.017997,4.055908,2.147450
221,nutrition5k/imagery/overhead/dish_1560454488/r...,nutrition5k/imagery/sideA/dish_1560454488/firs...,nutrition5k/imagery/sideB/dish_1560454488/firs...,nutrition5k/imagery/sideC/dish_1560454488/firs...,nutrition5k/imagery/sideD/dish_1560454488/firs...,48.244003,42.0,4.500726,1.893843,0.635490


In [ ]:
#Download the folder
!zip -r nutrition5k.zip /content/nutrition5k
from google.colab import files
files.download('nutrition5k.zip')


  adding: content/nutrition5k/ (stored 0%)
  adding: content/nutrition5k/metadata/ (stored 0%)
  adding: content/nutrition5k/metadata/dish_metadata_cafe1.csv (deflated 67%)
  adding: content/nutrition5k/imagery/ (stored 0%)
  adding: content/nutrition5k/imagery/sideD/ (stored 0%)
  adding: content/nutrition5k/imagery/sideD/dish_1562775488/ (stored 0%)
  adding: content/nutrition5k/imagery/sideD/dish_1562775488/first_frame_camera_D.png (deflated 1%)
  adding: content/nutrition5k/imagery/sideD/dish_1562775488/camera_D.h264 (deflated 0%)
  adding: content/nutrition5k/imagery/sideD/dish_1558461431/ (stored 0%)
  adding: content/nutrition5k/imagery/sideD/dish_1558461431/first_frame_camera_D.png (deflated 0%)
  adding: content/nutrition5k/imagery/sideD/dish_1558461431/camera_D.h264 (deflated 0%)
  adding: content/nutrition5k/imagery/sideD/dish_1562862493/ (stored 0%)
  adding: content/nutrition5k/imagery/sideD/dish_1562862493/first_frame_camera_D.png (deflated 1%)
  adding: content/nutrition

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
!cp nutrition5k.zip /content/drive/MyDrive/D340

^C


In [3]:
from google.colab import files

files.download('nutrition5k.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Download the df_data as a csv file
df_merged.to_csv('df_merged.csv', index=False)
files.download('df_merged.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##**Data Generator**

In [11]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size, img_size, output, augmentor=None, shuffle=True):
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size = img_size
        self.augmentor = augmentor
        self.shuffle = shuffle
        self.on_epoch_end()
        self.output = output

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_df = self.df.iloc[index*self.batch_size:(index+1)*self.batch_size]

        overheads = []
        sideAs = []
        sideBs = []
        sideCs = []
        sideDs = []
        labels = []

        for _, row in batch_df.iterrows():
            overhead = tf.keras.preprocessing.image.load_img(row['Filepath'], target_size=self.img_size)
            sideA = tf.keras.preprocessing.image.load_img(row['sideAPath'], target_size=self.img_size)
            sideB = tf.keras.preprocessing.image.load_img(row['sideBPath'], target_size=self.img_size)
            sideC = tf.keras.preprocessing.image.load_img(row['sideCPath'], target_size=self.img_size)
            sideD = tf.keras.preprocessing.image.load_img(row['sideDPath'], target_size=self.img_size)

            overhead = tf.keras.preprocessing.image.img_to_array(overhead) / 255.0
            sideA = tf.keras.preprocessing.image.img_to_array(sideA) / 255.0
            sideB = tf.keras.preprocessing.image.img_to_array(sideB) / 255.0
            sideC = tf.keras.preprocessing.image.img_to_array(sideC) / 255.0
            sideD = tf.keras.preprocessing.image.img_to_array(sideD) / 255.0

            if self.augmentor:
                overhead = self.augmentor.random_transform(overhead)
                sideA = self.augmentor.random_transform(sideA)
                sideB = self.augmentor.random_transform(sideB)
                sideC = self.augmentor.random_transform(sideC)
                sideD = self.augmentor.random_transform(sideD)

            overheads.append(overhead)
            sideAs.append(sideA)
            sideBs.append(sideB)
            sideCs.append(sideC)
            sideDs.append(sideD)
            labels.append(row['protein'])

        return {
    "input_overhead": np.array(overheads),
    "input_sideA": np.array(sideAs),
    "input_sideB": np.array(sideBs),
    "input_sideC": np.array(sideCs),
    "input_sideD": np.array(sideDs)
}, np.array(labels)


    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


##**Create the Multi-input Models**

In [ ]:
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(120, 120, 3))

for layer in model_vgg16.layers:
    layer.trainable = False

# Input Layers
input_overhead = layers.Input(shape=(120, 120, 3), name="input_overhead")
input_sideA = layers.Input(shape=(120, 120, 3), name="input_sideA")
input_sideB = layers.Input(shape=(120, 120, 3), name="input_sideB")
input_sideC = layers.Input(shape=(120, 120, 3), name="input_sideC")
input_sideD = layers.Input(shape=(120, 120, 3), name="input_sideD")

# extract features
x_overhead = model_vgg16(input_overhead)
x_sideA = model_vgg16(input_sideA)
x_sideB = model_vgg16(input_sideB)
x_sideC = model_vgg16(input_sideC)
x_sideD = model_vgg16(input_sideD)


# flatten
x_overhead = layers.Flatten()(x_overhead)
x_sideA = layers.Flatten()(x_sideA)
x_sideB = layers.Flatten()(x_sideB)
x_sideC = layers.Flatten()(x_sideC)
x_sideD = layers.Flatten()(x_sideD)


# concatenate
merged = layers.concatenate([x_overhead, x_sideA, x_sideB, x_sideC, x_sideD])

# dropot and dense layers
x = layers.Dropout(0.5)(merged)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# output layer
# output = layers.Dense(1, activation='linear')(x)


# output for protein
output_protein = layers.Dense(1, activation='linear')(x)
model_protein = Model(inputs={
                            "input_overhead": input_overhead,
                            "input_sideA": input_sideA,
                            "input_sideB": input_sideB,
                            "input_sideC": input_sideC,
                            "input_sideD": input_sideD
                        }, outputs=output_protein)
model_protein.compile(optimizer='adam', loss='mse', metrics=['mae'])




##**Train Test Split**

In [13]:
train_val_df, test_df = train_test_split(df_data, train_size=0.7, shuffle=True, random_state=1)
train_df, val_df = train_test_split(train_val_df, train_size=0.8, shuffle=True, random_state=1)

In [14]:
data_augmentor = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
)

##**Protein**

In [12]:
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(120, 120, 3))

for layer in model_vgg16.layers:
    layer.trainable = False

# Input Layers
input_overhead = layers.Input(shape=(120, 120, 3), name="input_overhead")
input_sideA = layers.Input(shape=(120, 120, 3), name="input_sideA")
input_sideB = layers.Input(shape=(120, 120, 3), name="input_sideB")
input_sideC = layers.Input(shape=(120, 120, 3), name="input_sideC")
input_sideD = layers.Input(shape=(120, 120, 3), name="input_sideD")

# extract features
x_overhead = model_vgg16(input_overhead)
x_sideA = model_vgg16(input_sideA)
x_sideB = model_vgg16(input_sideB)
x_sideC = model_vgg16(input_sideC)
x_sideD = model_vgg16(input_sideD)


# flatten
x_overhead = layers.Flatten()(x_overhead)
x_sideA = layers.Flatten()(x_sideA)
x_sideB = layers.Flatten()(x_sideB)
x_sideC = layers.Flatten()(x_sideC)
x_sideD = layers.Flatten()(x_sideD)


# concatenate
merged = layers.concatenate([x_overhead, x_sideA, x_sideB, x_sideC, x_sideD])

# dropot and dense layers
x = layers.Dropout(0.5)(merged)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# output layer
# output = layers.Dense(1, activation='linear')(x)


# output for protein
output_protein = layers.Dense(1, activation='linear')(x)
model_protein = Model(inputs={
                            "input_overhead": input_overhead,
                            "input_sideA": input_sideA,
                            "input_sideB": input_sideB,
                            "input_sideC": input_sideC,
                            "input_sideD": input_sideD
                        }, outputs=output_protein)
model_protein.compile(optimizer='adam', loss='mse', metrics=['mae'])




58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [15]:
# train the model

train_gen = DataGenerator(train_df, batch_size=32, img_size=(120, 120), output='protein', augmentor=data_augmentor)
val_gen = DataGenerator(val_df, batch_size=32, img_size=(120, 120), output='protein', augmentor=data_augmentor)

histories_protein = model_protein.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 203s 51s/step - loss: 655.4285 - mae: 17.1052 - val_loss: 267.8770 - val_mae: 13.0097
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 171s 43s/step - loss: 375.5935 - mae: 13.2331 - val_loss: 255.7872 - val_mae: 11.4195
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 206s 45s/step - loss: 324.6815 - mae: 11.6995 - val_loss: 247.4533 - val_mae: 13.4197
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 170s 42s/step - loss: 260.7617 - mae: 12.4415 - val_loss: 223.8581 - val_mae: 11.8944
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 205s 44s/step - loss: 195.4444 - mae: 9.5528 - val_loss: 261.6651 - val_mae: 11.5062
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 177s 44s/step - loss: 274.8326 - mae: 10.6132 - val_loss: 199.5476 - val_mae: 11.0959
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 181s 47s/step - loss: 205.1633 - mae: 9.8563 - val_loss: 215.0225 - val_mae: 11.0779
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 168s 43s/step - loss: 326.7346 - mae: 11.6129 - val_loss: 230.1860 - val_mae: 11.0924
Epoch 9/100
4/4 ━━

In [16]:
test_protein_gen = DataGenerator(
    test_df,
    batch_size=32,
    img_size=(120,120),
    output='protein',
    shuffle=False,

)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='protein',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

# predict
y_pred = model_protein.predict(test_protein_gen)
y_true = test_images.labels

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")



Found 67 validated image filenames.
3/3 ━━━━━━━━━━━━━━━━━━━━ 77s 17s/step
MSE: 286.4886
RMSE: 16.9260
MAE: 11.9059
R²: 0.4209


In [ ]:
# train the model

train_gen = DataGenerator(train_df, batch_size=32, img_size=(120, 120), output='protein', augmentor=data_augmentor)
val_gen = DataGenerator(val_df, batch_size=32, img_size=(120, 120), output='protein', augmentor=data_augmentor)

histories_protein = model_protein.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 86s 52s/step - loss: 457.2592 - mae: 16.4988 - val_loss: 324.4776 - val_mae: 15.1705
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 71s 41s/step - loss: 236.7290 - mae: 12.3557 - val_loss: 303.2026 - val_mae: 14.5415
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 72s 35s/step - loss: 205.1149 - mae: 10.8594 - val_loss: 298.0995 - val_mae: 13.4635
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 80s 34s/step - loss: 234.8104 - mae: 10.9353 - val_loss: 251.9539 - val_mae: 12.6131
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 83s 36s/step - loss: 213.2372 - mae: 11.0372 - val_loss: 230.3353 - val_mae: 12.1985
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 68s 38s/step - loss: 170.8481 - mae: 10.1048 - val_loss: 176.1297 - val_mae: 10.6730
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 70s 41s/step - loss: 178.1067 - mae: 10.6207 - val_loss: 179.1164 - val_mae: 10.2712
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 70s 35s/step - loss: 144.2639 - mae: 9.7652 - val_loss: 188.6571 - val_mae: 10.6976
Epoch 9/100
2/2 ━━━━━━━━━

In [ ]:
test_protein_gen = DataGenerator(
    test_df,
    batch_size=32,
    img_size=(120,120),
    output='protein',
    shuffle=False,

)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='protein',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

# predict
y_pred = model_protein.predict(test_protein_gen)
y_true = test_images.labels

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")



Found 32 validated image filenames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 40s 40s/step
MSE: 250.3667
RMSE: 15.8230
MAE: 11.0712
R²: 0.4005


##**Carb**

In [ ]:
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(120, 120, 3))

for layer in model_vgg16.layers:
    layer.trainable = False

# Input Layers
input_overhead = layers.Input(shape=(120, 120, 3), name="input_overhead")
input_sideA = layers.Input(shape=(120, 120, 3), name="input_sideA")
input_sideB = layers.Input(shape=(120, 120, 3), name="input_sideB")
input_sideC = layers.Input(shape=(120, 120, 3), name="input_sideC")
input_sideD = layers.Input(shape=(120, 120, 3), name="input_sideD")

# extract features
x_overhead = model_vgg16(input_overhead)
x_sideA = model_vgg16(input_sideA)
x_sideB = model_vgg16(input_sideB)
x_sideC = model_vgg16(input_sideC)
x_sideD = model_vgg16(input_sideD)


# flatten
x_overhead = layers.Flatten()(x_overhead)
x_sideA = layers.Flatten()(x_sideA)
x_sideB = layers.Flatten()(x_sideB)
x_sideC = layers.Flatten()(x_sideC)
x_sideD = layers.Flatten()(x_sideD)


# concatenate
merged = layers.concatenate([x_overhead, x_sideA, x_sideB, x_sideC, x_sideD])

# dropot and dense layers
x = layers.Dropout(0.65)(merged)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.65)(x)

# output layer
# output = layers.Dense(1, activation='linear')(x)


# output for carb
output_carb = layers.Dense(1, activation='linear')(x)
model_carb = Model(inputs={
                            "input_overhead": input_overhead,
                            "input_sideA": input_sideA,
                            "input_sideB": input_sideB,
                            "input_sideC": input_sideC,
                            "input_sideD": input_sideD
                        }, outputs=output_carb)
model_carb.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# train the model
train_gen = DataGenerator(train_df, batch_size=32, img_size=(120,120), output='carb', augmentor=data_augmentor)
val_gen = DataGenerator(val_df, batch_size=32, img_size=(120,120), output='carb', augmentor=data_augmentor)
model_carb.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 89s 45s/step - loss: 479.7033 - mae: 15.3823 - val_loss: 345.5479 - val_mae: 15.6262
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 79s 48s/step - loss: 266.0430 - mae: 13.2843 - val_loss: 333.7178 - val_mae: 14.0043
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 73s 36s/step - loss: 256.3576 - mae: 11.3272 - val_loss: 349.6146 - val_mae: 13.9286
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 77s 41s/step - loss: 322.8431 - mae: 12.7418 - val_loss: 291.3949 - val_mae: 12.7074
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 73s 37s/step - loss: 204.1660 - mae: 10.6669 - val_loss: 240.8567 - val_mae: 12.1963
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 76s 45s/step - loss: 238.5441 - mae: 11.3585 - val_loss: 226.9111 - val_mae: 11.8144
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 72s 41s/step - loss: 183.0093 - mae: 10.2236 - val_loss: 215.5924 - val_mae: 11.3994
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 81s 41s/step - loss: 112.9136 - mae: 8.1466 - val_loss: 209.4094 - val_mae: 10.7474
Epoch 9/100
2/2 ━━━━━━━━━

In [ ]:
test_carb_gen = DataGenerator(
    test_df,
    batch_size=32,
    img_size=(120,120),
    output='carb',
    shuffle=False,

)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='carb',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)


# predict
y_pred = model_carb.predict(test_carb_gen)
y_true = test_images.labels

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

Found 32 validated image filenames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step
MSE: 274.9227
RMSE: 16.5808
MAE: 12.6074
R²: 0.2408


##**fat**

In [ ]:
# train the model
train_gen = DataGenerator(train_df, batch_size=32, img_size=(120, 120), output='fat', augmentor=data_augmentor)
val_gen = DataGenerator(val_df, batch_size=32, img_size=(120, 120), output='fat', augmentor=data_augmentor)
model_fat.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 82s 39s/step - loss: 246.3592 - mae: 10.5110 - val_loss: 237.7742 - val_mae: 12.6800
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 74s 39s/step - loss: 186.6716 - mae: 11.0619 - val_loss: 206.5704 - val_mae: 11.8441
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 73s 44s/step - loss: 131.0064 - mae: 9.3951 - val_loss: 181.6186 - val_mae: 10.6702
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 74s 44s/step - loss: 117.6711 - mae: 8.0470 - val_loss: 198.7516 - val_mae: 10.3733
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 68s 39s/step - loss: 132.2075 - mae: 7.9401 - val_loss: 203.1184 - val_mae: 9.7795
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 66s 32s/step - loss: 109.2684 - mae: 7.2001 - val_loss: 230.3604 - val_mae: 10.0505
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 82s 32s/step - loss: 103.8735 - mae: 7.2527 - val_loss: 151.0649 - val_mae: 8.9624
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 67s 33s/step - loss: 65.9725 - mae: 5.6307 - val_loss: 146.1565 - val_mae: 8.8701
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━

In [ ]:
test_fat_gen = DataGenerator(
    test_df,
    batch_size=32,
    img_size=(120,120),
    output='fat',
    shuffle=False,

)
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='fat',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)
# predict
y_pred = model_fat.predict(test_fat_gen)
y_true = test_images.labels

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

Found 32 validated image filenames.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 35s 35s/step
MSE: 228.5422
RMSE: 15.1176
MAE: 11.9671
R²: 0.1391


##**mass**

In [17]:
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(120, 120, 3))

for layer in model_vgg16.layers:
    layer.trainable = False

# Input Layers
input_overhead = layers.Input(shape=(120, 120, 3), name="input_overhead")
input_sideA = layers.Input(shape=(120, 120, 3), name="input_sideA")
input_sideB = layers.Input(shape=(120, 120, 3), name="input_sideB")
input_sideC = layers.Input(shape=(120, 120, 3), name="input_sideC")
input_sideD = layers.Input(shape=(120, 120, 3), name="input_sideD")

# extract features
x_overhead = model_vgg16(input_overhead)
x_sideA = model_vgg16(input_sideA)
x_sideB = model_vgg16(input_sideB)
x_sideC = model_vgg16(input_sideC)
x_sideD = model_vgg16(input_sideD)


# flatten
x_overhead = layers.Flatten()(x_overhead)
x_sideA = layers.Flatten()(x_sideA)
x_sideB = layers.Flatten()(x_sideB)
x_sideC = layers.Flatten()(x_sideC)
x_sideD = layers.Flatten()(x_sideD)


# concatenate
merged = layers.concatenate([x_overhead, x_sideA, x_sideB, x_sideC, x_sideD])

# dropot and dense layers
x = layers.Dropout(0.5)(merged)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# output layer
# output = layers.Dense(1, activation='linear')(x)


# output for protein
output_mass = layers.Dense(1, activation='linear')(x)
model_mass = Model(inputs={
                            "input_overhead": input_overhead,
                            "input_sideA": input_sideA,
                            "input_sideB": input_sideB,
                            "input_sideC": input_sideC,
                            "input_sideD": input_sideD
                        }, outputs=output_mass)
model_mass.compile(optimizer='adam', loss='mse', metrics=['mae'])




In [18]:
# train the model
train_gen = DataGenerator(train_df, batch_size=32, img_size=(120,120), output='mass', augmentor=data_augmentor)
val_gen = DataGenerator(val_df, batch_size=32, img_size=(120,120), output='mass', augmentor=data_augmentor)
model_mass.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 191s 50s/step - loss: 541.9896 - mae: 18.4009 - val_loss: 249.1663 - val_mae: 12.8024
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 176s 45s/step - loss: 335.6599 - mae: 12.7346 - val_loss: 275.5893 - val_mae: 11.7286
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 172s 44s/step - loss: 264.6121 - mae: 11.6155 - val_loss: 223.8545 - val_mae: 12.2066
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 168s 42s/step - loss: 276.0561 - mae: 12.3319 - val_loss: 239.9098 - val_mae: 12.2822
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 170s 44s/step - loss: 305.0724 - mae: 12.0737 - val_loss: 225.4669 - val_mae: 11.7178
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 176s 45s/step - loss: 189.4900 - mae: 9.5509 - val_loss: 220.6846 - val_mae: 10.6559
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 173s 44s/step - loss: 156.7696 - mae: 9.0626 - val_loss: 191.0541 - val_mae: 10.8275
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 221s 47s/step - loss: 192.0932 - mae: 9.7629 - val_loss: 217.9113 - val_mae: 11.5390
Epoch 9/100
4/4 ━━━

In [20]:
test_mass_gen = DataGenerator(
    test_df,
    batch_size=32,
    img_size=(120,120),
    output='mass',
    shuffle=False,

)

# predict
y_pred = model_mass.predict(test_mass_gen)
y_true = test_images.labels

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 79s 16s/step
MSE: 298.5347
RMSE: 17.2782
MAE: 12.5961
R²: 0.3966


##**Calories**

In [21]:
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(120, 120, 3))

for layer in model_vgg16.layers:
    layer.trainable = False

# Input Layers
input_overhead = layers.Input(shape=(120, 120, 3), name="input_overhead")
input_sideA = layers.Input(shape=(120, 120, 3), name="input_sideA")
input_sideB = layers.Input(shape=(120, 120, 3), name="input_sideB")
input_sideC = layers.Input(shape=(120, 120, 3), name="input_sideC")
input_sideD = layers.Input(shape=(120, 120, 3), name="input_sideD")

# extract features
x_overhead = model_vgg16(input_overhead)
x_sideA = model_vgg16(input_sideA)
x_sideB = model_vgg16(input_sideB)
x_sideC = model_vgg16(input_sideC)
x_sideD = model_vgg16(input_sideD)


# flatten
x_overhead = layers.Flatten()(x_overhead)
x_sideA = layers.Flatten()(x_sideA)
x_sideB = layers.Flatten()(x_sideB)
x_sideC = layers.Flatten()(x_sideC)
x_sideD = layers.Flatten()(x_sideD)


# concatenate
merged = layers.concatenate([x_overhead, x_sideA, x_sideB, x_sideC, x_sideD])

# dropot and dense layers
x = layers.Dropout(0.5)(merged)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# output layer
# output = layers.Dense(1, activation='linear')(x)


# output for protein
output_calories = layers.Dense(1, activation='linear')(x)
model_calories = Model(inputs={
                            "input_overhead": input_overhead,
                            "input_sideA": input_sideA,
                            "input_sideB": input_sideB,
                            "input_sideC": input_sideC,
                            "input_sideD": input_sideD
                        }, outputs=output_calories)
model_calories.compile(optimizer='adam', loss='mse', metrics=['mae'])




In [23]:
# train the model
train_gen = DataGenerator(train_df, batch_size=32, img_size=(120,120), output='calories', augmentor=data_augmentor)
val_gen = DataGenerator(val_df, batch_size=32, img_size=(120,120), output='calories', augmentor=data_augmentor)
model_calories.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 185s 46s/step - loss: 713.7458 - mae: 18.3020 - val_loss: 262.3077 - val_mae: 12.8815
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 177s 46s/step - loss: 463.3014 - mae: 14.1021 - val_loss: 258.0362 - val_mae: 11.7941
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 197s 44s/step - loss: 247.3231 - mae: 10.9279 - val_loss: 221.3906 - val_mae: 11.8735
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 197s 42s/step - loss: 345.7925 - mae: 12.7176 - val_loss: 216.8463 - val_mae: 12.2087
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 173s 46s/step - loss: 307.5782 - mae: 11.8323 - val_loss: 208.0766 - val_mae: 10.6980
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 172s 44s/step - loss: 232.0353 - mae: 10.1749 - val_loss: 212.0957 - val_mae: 10.9326
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 173s 44s/step - loss: 196.3807 - mae: 9.4657 - val_loss: 226.7975 - val_mae: 11.3975
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 165s 42s/step - loss: 184.8049 - mae: 10.1122 - val_loss: 197.0785 - val_mae: 10.4477
Epoch 9/100
4/4 ━

KeyboardInterrupt: 

In [24]:
test_calories_gen = DataGenerator(
    test_df,
    batch_size=32,
    img_size=(120,120),
    output='calories',
    shuffle=False,

)

# predict
y_pred = model_calories.predict(test_calories_gen)
y_true = test_images.labels

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

2/3 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step 

3/3 ━━━━━━━━━━━━━━━━━━━━ 76s 16s/step
MSE: 297.7877
RMSE: 17.2565
MAE: 12.9393
R²: 0.3981
